# 📘 Архитектуры Агентов 11: Meta-Controller (Мета-Контроллер)

Добро пожаловать в одиннадцатый блокнот нашей серии. Сегодня мы создаем **Meta-Controller**, архитектуру управляющего агента, который оркестрирует команду специализированных субагентов. Этот паттерн является фундаментальным для создания мощных, многопрофильных систем ИИ.

Вместо создания единого монолитного агента, который пытается делать все, Meta-Controller действует как умный диспетчер. Он принимает входящий запрос, анализирует его природу и маршрутизирует его наиболее подходящему специалисту из пула доступных агентов. Это позволяет каждому субагенту быть высоко оптимизированным для своей конкретной задачи, что ведет к лучшей производительности и модульности.

Мы продемонстрируем это, создав систему с тремя различными специалистами:
1.  **Generalist Agent (Агент-Универсал):** Обрабатывает обычную беседу и простые вопросы.
2.  **Research Agent (Агент-Исследователь):** Оснащен инструментом поиска для ответов на вопросы о недавних событиях или сложных темах.
3.  **Coding Agent (Агент-Программист):** Специалист, сфокусированный на генерации фрагментов кода Python.

**Meta-Controller** будет "мозгом" операции, изучая каждый запрос пользователя и решая, какой агент лучше всего подходит для ответа. Это создает гибкую и легко расширяемую систему, где новые возможности могут быть добавлены простым созданием нового агента-специалиста и обучением контроллера о нем.

### Определение
**Meta-Controller** (или Маршрутизатор / Router) — это управляющий агент в мультиагентной системе, который отвечает за анализ входящих задач и отправку их соответствующему специализированному субагенту или рабочему процессу. Он действует как интеллектуальный слой маршрутизации, решая, какой инструмент или эксперт лучше всего подходит для текущей работы.

### Высокоуровневый рабочий процесс

1.  **Получение ввода (Receive Input):** Система получает запрос пользователя.
2.  **Анализ Meta-Controller'а:** Агент Meta-Controller изучает намерение запроса, сложность и содержание.
3.  **Отправка специалисту (Dispatch to Specialist):** На основе своего анализа Meta-Controller выбирает лучшего агента-специалиста (например, 'Researcher', 'Coder', 'Chatbot') из предопределенного пула.
4.  **Выполнение задачи (Execute Task):** Выбранный агент-специалист выполняет задачу и генерирует результат.
5.  **Возврат результата (Return Result):** Результат от специалиста возвращается пользователю. В более сложных рабочих процессах управление может вернуться к Meta-Controller для дальнейших шагов или мониторинга.

### Когда использовать / Применения
*   **Мульти-сервисные платформы ИИ:** Единая точка входа для платформы, предлагающей разнообразные услуги, такие как анализ документов, визуализация данных и творческое письмо.
*   **Адаптивные персональные ассистенты:** Ассистент, который может переключаться между различными режимами или инструментами, такими как управление календарем, поиск в интернете или управление устройствами умного дома.
*   **Корпоративные рабочие процессы:** Маршрутизация тикетов поддержки клиентов в правильный отдел (технический, биллинг, продажи) на основе содержания тикета.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Гибкость и Модульность:** Чрезвычайно легко добавлять новые возможности, просто добавляя нового агента-специалиста и обновляя логику маршрутизации контроллера.
    *   **Производительность:** Позволяет использовать высоко оптимизированных агентов-экспертов вместо одной модели "мастер на все руки", которая может быть посредственной во всем.
*   **Слабые стороны:**
    *   **Контроллер как единая точка отказа:** Качество всей системы зависит от способности контроллера правильно маршрутизировать задачи. Плохое решение по маршрутизации ведет к субоптимальному или неверному результату.
    *   **Потенциал для увеличенной задержки:** Дополнительный шаг маршрутизации может добавить небольшую задержку по сравнению с прямым вызовом одного агента.

## Фаза 0: Основа и Настройка

Мы установим библиотеки и настроим нашу среду. Нам понадобится `langchain-tavily` для инструмента поиска нашего Research Agent.

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Dict, Any
from dotenv import load_dotenv

# Pydantic для структурированного вывода маршрутизатора
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Meta-Controller (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

In [ ]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Meta-Controller (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Создание Агентов-Специалистов

Сначала мы создадим нашу команду агентов-экспертов. Каждый из них будет простой цепочкой (chain) с конкретной персоной и, в случае с Researcher (Исследователем), инструментом. Мы обернем их в функцию узла для использования в нашем LangGraph.

In [ ]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=3)

# Определение состояния для общего графа
class MetaAgentState(TypedDict):
    user_request: str
    next_agent_to_call: Optional[str]
    generation: str

# Фабричная функция-помощник для создания узлов агентов-специалистов
def create_specialist_node(persona: str, tools: list = None):
    """Фабрика для создания узла агента-специалиста."""
    system_prompt = f"You are a specialist agent with the following persona: {persona}. Respond directly and concisely to the user's request based on your role."
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{user_request}")
    ])
    
    if tools:
        chain = prompt | llm.bind_tools(tools)
    else:
        chain = prompt | llm
        
    def specialist_node(state: MetaAgentState) -> Dict[str, Any]:
        result = chain.invoke({"user_request": state['user_request']})
        return {"generation": result.content}
    
    return specialist_node

# 1. Узел Агента-Универсала (Generalist Agent Node)
generalist_node = create_specialist_node(
    "You are a friendly and helpful generalist AI assistant. You handle casual conversation and simple questions."
)

# 2. Узел Агента-Исследователя (Research Agent Node)
research_agent_node = create_specialist_node(
    "You are an expert researcher. You must use your search tool to find information to answer the user's question.",
    tools=[search_tool]
)

# 3. Узел Агента-Кодера (Coding Agent Node)
coding_agent_node = create_specialist_node(
    "You are an expert Python programmer. Your task is to write clean, efficient Python code based on the user's request. Provide only the code, wrapped in markdown code blocks, with minimal explanation."
)

print("Агенты-специалисты успешно определены.")

Specialist agents defined successfully.


## Фаза 2: Создание Meta-Controller

Это мозг нашей системы. Meta-Controller — это узел на базе LLM, чья единственная работа — решать, какому специалисту направить запрос. Качество его промпта критически важно для производительности системы.

In [ ]:
# Pydantic модель для решения контроллера о маршрутизации
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="Имя агента-специалиста для вызова следующим. Должно быть одним из ['Generalist', 'Researcher', 'Coder'].")
    reasoning: str = Field(description="Краткая причина выбора следующего агента.")

def meta_controller_node(state: MetaAgentState) -> Dict[str, Any]:
    """Центральный контроллер, который решает, какого специалиста вызвать."""
    console.print("--- 🧠 Meta-Controller Анализирует Запрос ---")
    
    # Определение специалистов и их описаний для контроллера
    specialists = {
        "Generalist": "Handles casual conversation, greetings, and simple questions.",
        "Researcher": "Answers questions about recent events, complex topics, or anything requiring up-to-date information from the web.",
        "Coder": "Writes Python code based on a user's specification."
    }
    
    specialist_descriptions = "\n".join([f"- {name}: {desc}" for name, desc in specialists.items()])
    
    prompt = ChatPromptTemplate.from_template(
        f"""You are the meta-controller for a multi-agent AI system. Your job is to analyze the user's request and route it to the most appropriate specialist agent.

Here are the available specialists:
{specialist_descriptions}

Analyze the following user request and choose the best specialist to handle it. Provide your decision in the required format.

User Request: "{{user_request}}""""
    )
    
    controller_llm = llm.with_structured_output(ControllerDecision)
    chain = prompt | controller_llm
    
    decision = chain.invoke({"user_request": state['user_request']})
    console.print(f"[yellow]Решение о маршрутизации:[/yellow] Отправить к [bold]{decision.next_agent}[/bold]. [italic]Причина: {decision.reasoning}[/italic]")
    
    return {"next_agent_to_call": decision.next_agent}

print("Узел Meta-Controller успешно определен.")

Meta-Controller node defined successfully.


## Фаза 3: Сборка и Запуск Графа

Теперь мы используем LangGraph, чтобы связать все вместе. Граф начнется с Meta-Controller, и затем, на основе его решения, условное ребро (conditional edge) направит состояние в узел правильного специалиста. После работы специалиста граф закончится.

In [ ]:
# Построение графа
workflow = StateGraph(MetaAgentState)

# Добавление узлов для контроллера и каждого специалиста
workflow.add_node("meta_controller", meta_controller_node)
workflow.add_node("Generalist", generalist_node)
workflow.add_node("Researcher", research_agent_node)
workflow.add_node("Coder", coding_agent_node)

# Установка точки входа
workflow.set_entry_point("meta_controller")

# Определение условной логики маршрутизации
def route_to_specialist(state: MetaAgentState) -> str:
    """Читает решение контроллера и возвращает имя узла для маршрутизации."""
    return state["next_agent_to_call"]

workflow.add_conditional_edges(
    "meta_controller",
    route_to_specialist,
    {
        "Generalist": "Generalist",
        "Researcher": "Researcher",
        "Coder": "Coder"
    }
)

# После работы любого специалиста процесс завершается
workflow.add_edge("Generalist", END)
workflow.add_edge("Researcher", END)
workflow.add_edge("Coder", END)

meta_agent = workflow.compile()
print("Граф агента Meta-Controller успешно скомпилирован.")

Meta-Controller agent graph compiled successfully.


## Фаза 4: Демонстрация

Давайте протестируем наш Meta-Controller с различными промптами, чтобы увидеть, правильно ли он отправляет их нужному специалисту.

In [ ]:
def run_agent(query: str):
    result = meta_agent.invoke({"user_request": query})
    console.print("\n[bold]Финальный ответ (Final Response):[/bold]")
    console.print(Markdown(result['generation']))

# Тест 1: Должен быть направлен к Generalist (Универсалу)
console.print("--- 💬 Тест 1: Обычная беседа ---")
run_agent("Hello, how are you today?")

# Тест 2: Должен быть направлен к Researcher (Исследователю)
console.print("\n--- 🔬 Тест 2: Исследовательский вопрос ---")
run_agent("What were NVIDIA's latest financial results?")

# Тест 3: Должен быть направлен к Coder (Кодеру)
console.print("\n--- 💻 Тест 3: Запрос на код ---")
run_agent("Can you write me a python function to calculate the nth fibonacci number?")

--- 💬 Test 1: General Conversation ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Generalist. Reason: The user's request is a simple greeting, which falls under the category of casual conversation handled by the Generalist agent.



Final Response:


Hello there! How can I help you today?


--- 🔬 Test 2: Research Question ---


...

NVIDIA's latest financial results, for the quarter ending in April 2024, were exceptionally strong. They reported revenue of $26.04 billion, a significant increase year-over-year, driven largely by their Data Center revenue which hit a record $22.6 billion. Their GAAP earnings per diluted share were $5.98.


--- 💻 Test 3: Coding Request ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Coder. Reason: The user is explicitly asking for a Python function, which is a coding task. The Coder agent is the specialist for this.



Final Response:


```python
def fibonacci(n):
    """Calculates the nth Fibonacci number."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b
```

## Заключение

В этом блокноте мы успешно реализовали архитектуру **Meta-Controller**. Наши тесты ясно демонстрируют её основную функцию: действие в качестве интеллектуального и динамического маршрутизатора.

1.  Простое приветствие было правильно идентифицировано и отправлено **Generalist'у**.
2.  Запрос о последних финансовых новостях был отправлен **Researcher'у**, который использовал свой инструмент поиска для получения актуальной информации.
3.  Запрос на фрагмент кода был направлен **Coder'у**, который предоставил хорошо отформатированную и корректную функцию.

Этот паттерн исключительно мощен для создания масштабируемых и поддерживаемых систем ИИ. Разделяя обязанности, каждый специалист может быть улучшен независимо, не затрагивая других. Общий интеллект системы может быть повышен простым добавлением новых, более способных специалистов и уведомлением Meta-Controller'а о них. Хотя сам контроллер представляет собой потенциальное "узкое место", его роль как гибкого оркестратора является краеугольным камнем продвинутого агентного дизайна.